In [5]:
!pip install blosc

In [6]:
import os
import numpy as np
import blosc
import pickle
from numpy import genfromtxt
import zmq
import json
import random

In [7]:
def zip_and_pickle(obj, flags=0, protocol=-1):
    """pickle an object, and zip the pickle before sending it"""
    p = pickle.dumps(obj, protocol)
    z = blosc.compress(p, typesize=8)
    return z

In [8]:
# For now this is only to get rows to be classified, not trained (with partial fit)
class Chunk_Generator(object):
    def __init__(self):
        pass
    
    def set_file_details(self, meas, exp, user):
        self.meas = meas
        self.exp = exp
        self.user = user
        
    def getrows(self, chunkrows):
        delimiter = " "
        header_names = ["x", "y", "z"]
        
        data_folder = 'data/raw_data'
        if int(self.exp) < 10:
            self.exp = str(self.exp).zfill(2)
        
        if int(self.user) < 10:
            self.user = str(self.user).zfill(2)
            
        if self.meas == 'both':
            filename = "acc_exp" + self.exp + "_user" + self.user + ".txt"
            path = data_folder + '/' + filename
            acc_data = genfromtxt(path, delimiter=' ')
            
            filename = "gyro_exp" + self.exp + "_user" + self.user + ".txt"
            path = data_folder + '/' + filename
            gyro_data = genfromtxt(path, delimiter=' ')
            return (acc_data[chunkrows], gyro_data[chunkrows])
        else:
            filename = self.meas + "_exp" + self.exp + "_user" + self.user + ".txt"
            path = data_folder + '/' + filename
            data = genfromtxt(path, delimiter=' ')
            return data[chunkrows]
    
    def iter_minibatches(self, chunksize, numtrainingpoints):
        # Provide chunks one by one
        chunkstartmarker = 0
        while chunkstartmarker < numtrainingpoints:
            chunkrows = range(chunkstartmarker, chunkstartmarker+chunksize)
            X_chunk = self.getrows(chunkrows)
#             print(np.average(X_chunk))
            yield X_chunk
            chunkstartmarker += chunksize

In [9]:
data_folder = 'data/raw_data'

# path joining version for other paths
no_of_files = (len([name for name in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, name))]))
print(no_of_files)

123


In [10]:
for i, filename in enumerate(sorted(os.listdir(data_folder))):
#     print(i, filename)
    pass

In [11]:
data_folder = 'data/raw_data'
filename = "acc_exp01_user01.txt"
path = data_folder + '/' + filename

my_data = genfromtxt(path, delimiter=' ')
print(my_data[0:10])

[[ 0.91805559 -0.1125      0.50972225]
 [ 0.91111113 -0.09305556  0.53750004]
 [ 0.8819445  -0.08611111  0.51388893]
 [ 0.8819445  -0.08611111  0.51388893]
 [ 0.87916671 -0.1         0.50555558]
 [ 0.88888896 -0.10555556  0.51250004]
 [ 0.86250001 -0.10138889  0.50972225]
 [ 0.86111112 -0.10416667  0.5013889 ]
 [ 0.85416666 -0.10833334  0.5277778 ]
 [ 0.85138888 -0.10138889  0.5527778 ]]


In [12]:
cg = Chunk_Generator()
cg.set_file_details('acc', 1, 1)

batcherator = cg.iter_minibatches(chunksize=10, numtrainingpoints=100)
aggregate = []
for X_chunk in batcherator:
#     print(X_chunk)
    aggregate.extend(X_chunk.tolist())

# print(aggregate)
    
# if np.equal(aggregate, my_data[0:100]).all():
#     print("Equal")
# else:
#     print("Not equal")

In [13]:
ZMQ_HOST = 'localhost'
ZMQ_PORT = 7000

In [18]:
socket = zmq.Context().socket(zmq.DEALER)
ident = str(random.randint(1, 5)).zfill(3)
socket.identity = u"Client-{}".format(ident).encode("ascii")
socket.connect("tcp://{}:{}".format(ZMQ_HOST, ZMQ_PORT))
print(socket.identity)

b'Client-001'


In [19]:
decode = lambda x: x.decode('utf-8')
encode = lambda x: x.encode('ascii')

In [20]:
meas = 'both'
cg = Chunk_Generator()
cg.set_file_details(meas, 1, 1)

batcherator = cg.iter_minibatches(chunksize=550, numtrainingpoints=20000)
aggregate = []

    
if meas == 'both':
    for acc_chunk, gyro_chunk in batcherator:
    #     aggregate.extend(X_chunk.tolist())
        dict_req = {}

        dict_req["sender"] = socket.identity.decode('ascii')
        dict_req["command"] = "CLASSIFY"
        dict_req["database"] = meas
        dict_req = encode(json.dumps(dict_req))

        pickled_acc_chunk = zip_and_pickle(acc_chunk)
        pickled_gyro_chunk = zip_and_pickle(gyro_chunk)

        request = [dict_req, pickled_acc_chunk, pickled_gyro_chunk]
        message = socket.send_multipart(request)
else:
    for X_chunk in batcherator:
    #     aggregate.extend(X_chunk.tolist())
        dict_req = {}
 
        dict_req["sender"] = socket.identity.decode('ascii')
        dict_req["command"] = "CLASSIFY"
        dict_req["database"] = meas
        dict_req = encode(json.dumps(dict_req))

        pickled_chunk = zip_and_pickle(X_chunk)

        request = [dict_req, pickled_chunk]
        message = socket.send_multipart(request)